In [1]:
## Generate a graph
using StatsBase
using Combinatorics

include("jl/omega.jl")
include("jl/HSBM.jl")
include("jl/hypergraph_louvain.jl")
include("jl/inference.jl");

In [2]:
n = 100
Z = rand(1:2, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

ω(p,α) = (10 .*μ*sum(p))^(-sum(p))*prod(p.^α)^(1/(sum(p)*α))
α0 = 1

kmax = 3

Ω = buildΩ(ω, α0, kmax)
## Sample
H = sampleSBM(Z, ϑ, Ω; α=α0, kmax=kmax, kmin = 1)

Ω (generic function with 1 method)

In [3]:
## Run Louvains, Naive, and Faster version

@time Zlou = Naive_HyperLouvain(H,Ω;α=α0)
@time Zlou2 = HyperLouvain(H,kmax,Ω;α=α0)

mlou = modularity(H,Zlou,Ω,α=α0)
mlou2 = modularity(H,Zlou2,Ω,α=α0)

mlou ≈ mlou2
mlou

UndefVarError: UndefVarError: H not defined

In [5]:
## Try supernode Louvain
# Sanity check: having a singletons initialization is equivalent to naive louvain
include("jl/hypergraph_louvain.jl")
Z = collect(1:n)
@time Znew, changed = Naive_SuperNodeStep(H,Z,kmax,Ω;α=α0)
mlou3 =  modularity(H,Znew,Ω,α=α0)

UndefVarError: UndefVarError: Naive_SuperNodeStep not defined

In [ ]:
## Now repeatedly call the Naive_SuperNodeStep until no more improvement is possible

@time Z = SuperNodeLouvain(H,kmax,Ω;α=α0)
msuper = modularity(H,Z,Ω,α=α0)